# Exemple AFIR

## Objectif 

Compléter l'exemple simple avec un jeu de données réel.

- réseau autoroutier : 'roads_GL2017_Council_FR'
- stations IRVE : 'csl_gireve202407.csv'

In [1]:
import os
import sys

#new_path = r"D:\philippe\python ESstandard\geonetworkx"
new_path = os.getcwd()[:-8]
sys.path.append(new_path)
from shapely import LineString, Point
import numpy as np
import geopandas as gpd
import pandas as pd
import geonetworkx as gnx 
import networkx as nx 

from geonetworkx import geom_to_crs, cast_id 

## Création du réseau routier

Le réseau est construit à partir d'un GeoDataFrame des troncons routier. Les noeuds sont déduits des valeurs du champ 'geometry' (extrémités de la LineString).

In [2]:
rtet = gpd.read_file("./roads_GL2017_Council_FR/roads_GL2017_Council_FR.shp").to_crs(2154)
gr = gnx.from_geopandas_edgelist(rtet, edge_attr=["GEO_LENGTH", "ID", "geometry"])

                  ID COUNTRY_CO MEMBER_STA  \
0           22500102         FR          1   
1           22500101         FR          1   
2           22500134         FR          1   
3           22500103         FR          1   
4           22500177         FR          1   
..               ...        ...        ...   
482         22500184         FR          1   
483  151008084937001         FR          1   
484  210302122851481         FR          1   
485  211015123914126         FR          1   
486  211015124533612         FR          1   

                                            DESCRIPTIO CORE_NETWO CORRIDORS  \
0    Artigues-près-Bordeaux (J. N89/N230) <--> Bègl...          1         G   
1    Lormont (J. A10/A630/N230) <--> Artigues-près-...          1         G   
2    Brie-Comte-Robert (J. N104/N19) <--> Pontault-...          1       GHL   
3    Villenave-d'Ornon (J. A630/A62) <--> Bègles (J...          1         G   
4    Balbigny (J. N82/A89) <--> La Tour-de-Salvagny.

In [3]:
refmap = {'tiles': 'cartodbpositron', 'location': [46.3, 2.3], 'zoom_start': 7}
gr.explore(refmap=refmap, n_color='red')

## initialisation des stations à inclure

In [4]:
# Chargement des points de recharge de la consolidation Gireve
csl = pd.read_csv("csl_gireve202407.csv", sep=";", encoding="latin")

#csl["puissance_nominale"] = csl["puissance_nominale"].str.replace(",", ".")
csl["puissance_nominale"] = csl["puissance_nominale"].astype(float)

# Les stations respectant les critères AFIR sont obtenues après un groupby sur les coordonnées
stations = csl.groupby("coordonneesXY").agg(
    p_max = ("puissance_nominale", "max"), 
    p_cum = ("puissance_nominale", "sum"),
    id_station = ("id_pdc_regroupement", "first"),
    amenageur = ("nom_amenageur", "first"),
    operateur = ("nom_operateur", "first"))
stations_afir = stations[(stations["p_max"] > 150) & (stations["p_cum"] > 300)].copy().reset_index()
stations_afir['geometry'] = stations_afir["coordonneesXY"].apply(lambda x: Point(str.split(x, ',')))
del stations_afir['coordonneesXY']
stations_afir = gpd.GeoDataFrame(stations_afir, crs=4326).to_crs(2154)
stations_afir['node_id'] = ["st" + str(idx) for idx in range(len(stations_afir))]
stations_afir

,p_max,p_cum,id_station,amenageur,operateur,geometry,node_id
0,350.0,4486.0,FR*IOY*E454209,IONITY GmbH,IONITY GmbH | FR*IOY,POINT (468432.303 6599052.993),st0
1,300.0,3144.0,FRALLEGO8000402,Allego B.V.,Allego B.V. | FR*ALL,POINT (468423.434 6599220.814),st1
2,200.0,672.0,FR*PD1*ENET*LRL*BBC20001*3,Power Dot France,Power Dot S.A. | FR*PD1,POINT (459295.276 6392264.649),st2
3,188.0,2666.0,FR*PD1*ELCL*LFL*KMP40002*8,Power Dot France,Power Dot S.A. | FR*PD1,POINT (470971.903 6737943.233),st3
4,300.0,2488.0,ES*ZUN*E1111ER6,Grupo Easycharger,Zunder | ES*ZUN,POINT (475215.54 6851345.888),st4
...,...,...,...,...,...,...,...
1839,300.0,2532.0,FR*LMS*E1234603121*2,ENGIE Vianeo,Threeforce B.V. | FR*LMS,POINT (1047527.475 6850636.276),st1839
1840,200.0,758.0,FR*SSD*E10932*P1,concessionnaire VOLVO,Driveco | FR*SSD,POINT (1048167.911 6846328.391),st1840
1841,250.0,4000.0,FR*TSL*E0005U9,TESLA MOTORS NETHERLANDS B.V.,Tesla Motors Netherlands B.V. | NL*TSL,POINT (1048344.012 6849678.046),st1841
1842,160.0,640.0,FR*BMP*E1454,Bump,Bump | FR*BMP,POINT (1050900.6 6840362.982),st1842


## filtrage des stations

On ne garde que les stations situées à moins d'une distance d'un noeud ou d'un tronçon routier.

In [7]:
proxi = 2000 # distance maximale des stations au réseau routier
noeuds = gr.to_geopandas_nodelist()
troncons = gr.to_geopandas_edgelist()

sj_noeuds = gpd.sjoin(stations_afir, noeuds.set_geometry(noeuds.buffer(proxi))) # filtrage des noeuds
print(len(sj_noeuds))
print(sj_noeuds)

stations_afir_noeuds = stations_afir[stations_afir.index.isin(sj_noeuds.index)]
#print(len(stations_afir_noeuds))

stations_afir_autre = stations_afir[~stations_afir.index.isin(sj_noeuds.index)]


sj_lin = gpd.sjoin(stations_afir_autre, troncons.set_geometry(troncons.buffer(proxi))) # filtrage des tronçons
stations_afir_lin = stations_afir_autre[stations_afir_autre.index.isin(sj_lin.index)]

#stations_afir_gr = pd.concat([stations_afir_lin, stations_afir_noeuds])

print('Nb stations (noeuds, lin, tot) : ', len(stations_afir_noeuds), len(stations_afir_lin), len(stations_afir))

1302
      p_max   p_cum              id_station                      amenageur  \
0     350.0  4486.0          FR*IOY*E454209                    IONITY GmbH   
1     300.0  3144.0         FRALLEGO8000402                    Allego B.V.   
4     300.0  2488.0         ES*ZUN*E1111ER6              Grupo Easycharger   
8     175.0  1338.0  FR*HPC*ENF080070*001*2                   total_fr_hpc   
9     350.0  1543.0          FR*IOY*E402151                    IONITY GmbH   
...     ...     ...                     ...                            ...   
1839  300.0  2532.0    FR*LMS*E1234603121*2                   ENGIE Vianeo   
1839  300.0  2532.0    FR*LMS*E1234603121*2                   ENGIE Vianeo   
1840  200.0   758.0        FR*SSD*E10932*P1          concessionnaire VOLVO   
1841  250.0  4000.0          FR*TSL*E0005U9  TESLA MOTORS NETHERLANDS B.V.   
1842  160.0   640.0            FR*BMP*E1454                           Bump   

                                     operateur  \
0       

In [8]:
node_attr = ['amenageur', 'p_cum', 'id_station']
edge_attr = {'type': 'st_irve'}
gs_noeuds, stations_afir_autre2 = gnx.project_graph(stations_afir, noeuds, proxi, node_attr, edge_attr)
print(gs_noeuds.to_geopandas_edgelist())
gs_lin = gnx.from_geopandas_nodelist(stations_afir_lin, node_id='node_id', node_attr=['amenageur', 'p_cum', 'id_station', 'node_id'])
stations = list(stations_afir_lin['node_id'])

for station in stations: # non vectorisé pour minimiser les noeuds supplémentaires
    dist = gs_lin.project_node(station, gr, proxi, edge_attr)
    if not dist:
        geo_st = gs_lin.nodes[station]['geometry'].centroid
        id_edge = gr.find_nearest_edge(geo_st, proxi) # recheche d'un troncon à moins de 3 km
        if not id_edge: 
            continue 
        # on ajoute un noeud et on crée le lien 
        id_node = int(max(cast_id(gr.nodes, only_int=True)) + 1)
        gr.insert_node(geo_st, id_node, id_edge, adjust=False) 
        dis1 = geo_st.distance(gr.nodes[id_node]['geometry'])
        geo1 = LineString([gr.nodes[id_node]['geometry'], geo_st])
        gs_lin.add_edge(id_node, station, **(edge_attr | {'geometry':geo1, 'weight': dis1})) # ajout du lien entre la station et le noeud routier
        # gs2.add_node(id_node, **(att_node | {'geometry': intersect}))
gs = gnx.compose(gs_noeuds, gs_lin)
gs.to_geopandas_edgelist()

      source  target       weight  \
0        st0   415.0    72.891242   
1        st1   415.0   100.652761   
4        st4   416.0   181.941449   
8        st8   417.0   173.127901   
9        st9   418.0    44.267347   
...      ...     ...          ...   
1838  st1838   957.0   220.834643   
1839  st1839    40.0  1216.445156   
1840  st1840   958.0    56.478203   
1841  st1841    40.0   759.184831   
1842  st1842   959.0   499.691764   

                                               geometry     type  
0     LINESTRING (468432.303 6599052.993, 468407.302...  st_irve  
1     LINESTRING (468423.434 6599220.814, 468407.302...  st_irve  
4     LINESTRING (475215.54 6851345.888, 475050.097 ...  st_irve  
8     LINESTRING (449751.625 6241587.556, 449846.563...  st_irve  
9     LINESTRING (463949.502 6706698.242, 463925.146...  st_irve  
...                                                 ...      ...  
1838  LINESTRING (1046355.359 6855648.545, 1046211.5...  st_irve  
1839  LINESTRING (1

,source,target,geometry,weight,type
0,st0,415.0,"LINESTRING (468432.303 6599052.993, 468407.302...",72.891242,st_irve
1,415.0,st1,"LINESTRING (468423.434 6599220.814, 468407.302...",100.652761,st_irve
2,st4,416.0,"LINESTRING (475215.54 6851345.888, 475050.097 ...",181.941449,st_irve
3,st8,417.0,"LINESTRING (449751.625 6241587.556, 449846.563...",173.127901,st_irve
4,st9,418.0,"LINESTRING (463949.502 6706698.242, 463925.146...",44.267347,st_irve
...,...,...,...,...,...
1193,st1838,957.0,"LINESTRING (1046355.359 6855648.545, 1046211.5...",220.834643,st_irve
1194,st1839,40.0,"LINESTRING (1047527.475 6850636.276, 1048708.3...",1216.445156,st_irve
1195,40.0,st1841,"LINESTRING (1048344.012 6849678.046, 1048708.3...",759.184831,st_irve
1196,st1840,958.0,"LINESTRING (1048167.911 6846328.391, 1048112.4...",56.478203,st_irve


In [7]:
gs_noeuds.to_geopandas_edgelist()

,source,target,geometry,type,weight
0,st24,196.0,"LINESTRING (461292.984 6901410.455, 459310.192...",st_irve,1996.972025
1,196.0,st25,"LINESTRING (461064.19 6901290.481, 459310.192 ...",st_irve,1790.066564
2,196.0,st26,"LINESTRING (459823.624 6901089.42, 459310.192 ...",st_irve,758.709933
3,st32,146.0,"LINESTRING (456800.494 6898765.449, 456448.095...",st_irve,504.969391
4,146.0,st37,"LINESTRING (455442.501 6899662.375, 456448.095...",st_irve,1139.171927
...,...,...,...,...,...
248,st1831,59.0,"LINESTRING (1046120.953 6836215.489, 1046325.6...",st_irve,417.931095
249,59.0,st1834,"LINESTRING (1046372.589 6835163.624, 1046325.6...",st_irve,689.076166
250,59.0,st1835,"LINESTRING (1046349.938 6836195.481, 1046325.6...",st_irve,345.242136
251,st1839,40.0,"LINESTRING (1047527.475 6850636.276, 1048708.3...",st_irve,1216.445156


## Création du réseau des stations

Le réseau ne comporte que les noeuds correspondant aux stations.

In [8]:
gs = gnx.from_geopandas_nodelist(stations_afir_gr, node_id='node_id', node_attr=['amenageur', 'p_cum', 'id_station', 'node_id'])
gs.to_geopandas_nodelist()

NameError: name 'stations_afir_gr' is not defined

## Projection du réseau des stations sur le réseau routier

Les stations sont associées au noeud du graphe routier le plus proche (s'il n'y en a pas à moins de 'proxi' km, un noeud est ajouté sur le troncon le plus proche).
Les deux réseaux peuvent être regroupés en un seul.

In [ ]:
stations = list(gs.nodes)
att_edges_station = {'type': 'st_irve'}

for station in stations:
    dist = gs.project_node(station, gr, proxi, att_edges_station)
    if not dist:
        geo_st = gs.nodes[station]['geometry'].centroid
        id_edge = gr.find_nearest_edge(geo_st, proxi) # recheche d'un troncon à moins de 3 km
        if not id_edge: 
            continue 
        # on ajoute un noeud et on crée le lien 
        id_node = max(cast_id(gr.nodes, only_int=True)) + 1
        gr.insert_node(geo_st, id_node, id_edge, adjust=False) 
        dis1 = geo_st.distance(gr.nodes[id_node]['geometry'])
        geo1 = LineString([gr.nodes[id_node]['geometry'], geo_st])
        gs.add_edge(id_node, station, **(att_edges_station | {'geometry':geo1, 'weight': dis1})) # ajout du lien entre la station et le noeud routier
    
gs.to_geopandas_edgelist()

La séparation en deux réseaux permet de les afficher avec des paramètres différents.

Dans cet exemple, les couleurs des noeuds et troncons sont noir / bleu pour le réseau routier et vert / gris pour le réseau des stations. Les contenus des 'popup' et 'tooltip' sont également différents. Chaque type d'élément est activable sur une couche spécifique.

In [ ]:
param_exp_gs = {'e_tooltip': ["source", "target"], 'e_popup': ['type', 'weight', 'source', 'target'], 'e_color': 'grey',
                'n_name': 'station', 'e_name': 'liaison station', 'n_popup': ['amenageur', 'p_cum', 'node_id', 'id_station'], 
                'n_tooltip': "amenageur", 'n_color': 'green', 'n_marker_kwds': {'radius': 4, 'fill': True}}
param_exp_gr = {'e_name': 'edges', 'n_name': 'nodes', 'e_popup': ['weight', 'source', 'target'], 'e_tooltip': ["source", "target"], 
                'n_tooltip': ["node_id"], 'n_marker_kwds': {'radius': 1, 'fill': False}}
carte = gs2.explore(refmap=refmap, **param_exp_gs)
carte = gr.explore(refmap=carte, layercontrol=True, n_color='black', **param_exp_gr)
carte

## Exemples d'utilisation du réseau global

In [ ]:
g_tot = gnx.compose(gr, gs) # réunion des deux graphes

### Trajet entre deux stations

In [ ]:
nx.shortest_path_length(g_tot, source='st1284', target='st1505', weight='weight')

In [ ]:
path = nx.shortest_path(g_tot, source='st1284', target='st1505', weight='weight')
#gr_path = g_tot.path_view(path)
gr_path = nx.subgraph_view(g_tot, filter_node=(lambda x: x in path))
path

In [ ]:
param_exp_path = {'e_name': 'path', 'e_color': 'red'}
refmap = {'tiles': 'cartodbpositron', 'location': [43.8, 5], 'zoom_start': 9}

carte = gs.explore(refmap=refmap, **param_exp_gs)
carte = gr.explore(refmap=carte, **param_exp_gr)
carte = gr_path.explore(refmap=carte, layercontrol=True, nodes=False, **param_exp_path)
carte

### Tronçons saturés

Identification des tronçons saturés (un tronçon est saturé s'il existe un point de ce tronçon situé à une distance de la plus proche station supérieure à un seuil)

In [ ]:
from time import time
import timeit

In [ ]:
#%%timeit
seuil = 50000
saturation = []
for edge in gr.edges:
    inter_st = g_tot.weight_extend(edge, gs, radius=seuil, n_attribute='dist_stat3')
    if not inter_st or inter_st > 2 * seuil :
        saturation.append(edge)

In [ ]:
print(saturation)

In [ ]:
gr_satur = nx.subgraph_view(g_tot, filter_edge=(lambda x1, x2: (x1, x2) in saturation))

In [ ]:
param_exp_path = {'e_name': 'saturation', 'e_color': 'red'}
refmap = {'tiles': 'cartodbpositron', 'location': [43.8, 5], 'zoom_start': 9}

carte = gs.explore(refmap=refmap, **param_exp_gs)
carte = gr.explore(refmap=carte, **param_exp_gr)
carte = gr_satur.explore(refmap=carte, layercontrol=True, nodes=False, **param_exp_path)
carte

## Troncons dont un des points est à plus de 50 km d'une station

calcul xa1 : 
    recherche de la station la plus proche du point A et du point B : distance xa et xb
saturation si max(dist-xa, 0) + max(dist-xb, 0) < weight(x)
    ou si seuil * 2 -xa -xb -wx < 0
    ou xa + xb + wx > 2 * seuil
    
xa1 > xa2

50-a > l-50 +b

max(50-a, 0) + max(50-b, 0) > l

ko si max(dist-weight(stat_adj1), 0) + max(dist-weight(stat_adj2), 0) < weight(node)


In [ ]:
%%timeit
gr_nd_df = gr.to_geopandas_nodelist()
gr_nd_df['dist'] = gr_nd_df['node_id'].apply(lambda x: g_tot.weight_node_to_graph(x, gs, radius=seuil))
#print(gr_nd_df[['node_id', 'dist']])
gr_ed_df = gr.to_geopandas_edgelist()
dist_s = gr_ed_df.merge(gr_nd_df, left_on='source', right_on='node_id', how='left')['dist']
dist_t = gr_ed_df.merge(gr_nd_df, left_on='target', right_on='node_id', how='left')['dist']
gr_ed_df